In [2]:
import torch
import torch.quantization.quantize_fx as quantize_fx
from model import BiConv
import copy

In [3]:
model_fp = BiConv()

Posting Training Dynamic/weight_only Quantization

In [5]:
model_to_quantize = copy.deepcopy(model_fp)
qconfig = torch.quantization.default_dynamic_qconfig
model_to_quantize.eval()
qconfig_dict = {"": qconfig}
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_dict)
model_quantized = quantize_fx.convert_fx(model_prepared)
# print(model_quantized)

input_fp32 = torch.randn(1, 1, 128, 128)
res = model_quantized(input_fp32)
print(res)

tensor([[-0.0355, -0.2913,  0.1903,  0.0765, -0.0081,  0.0220,  0.2269,  0.0348,
         -0.1663, -0.0268]])


Posting Training Static Quantization

In [7]:
model_to_quantize = copy.deepcopy(model_fp)
qconfig = torch.quantization.get_default_qconfig('qnnpack')
model_to_quantize.eval()
qconfig_dict = {"": qconfig}
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_dict)
model_quantized = quantize_fx.convert_fx(model_prepared)
# print(model_quantized)

input_fp32 = torch.randn(1, 1, 128, 128)
res = model_quantized(input_fp32)
print(res)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


Quantization Aware Training for Static Quantization

In [12]:
model_to_quantize = copy.deepcopy(model_fp)
qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')

model_to_quantize.train()
qconfig_dict = {"": qconfig}
model_prepared = quantize_fx.prepare_qat_fx(model_to_quantize, qconfig_dict)
model_quantized = quantize_fx.convert_fx(model_prepared)

input_fp32 = torch.randn(1, 1, 128, 128)
res = model_quantized(input_fp32)
print(res)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


Fusion

In [11]:
model_to_quantize = copy.deepcopy(model_fp)
model_to_quantize.eval()
model_fused = quantize_fx.fuse_fx(model_to_quantize)

input_fp32 = torch.randn(1, 1, 128, 128)
res = model_quantized(input_fp32)
print(res)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [67]:
from torch.quantization import QConfigMapping
model_fp32 = copy.deepcopy(model_fp)
qconfig_mapping = QConfigMapping.set_global(qconfig)
model_fp32.traceable_submodule = quantize_fx.prepare_fx(model_fp32.traceable_submodule, qconfig_mapping, input_fp32)

ImportError: cannot import name 'QConfigMapping' from 'torch.quantization' (/data_1/anaconda3/envs/yolov7/lib/python3.7/site-packages/torch/quantization/__init__.py)